In [ ]:
import pandas as pd 
import requests as rq 
from bs4 import BeautifulSoup



In [ ]:
raw_list = rq.get('https://www.assureurs.pro/entreprise_localisation/toutes-les-regions/nord-pas-de-calais/')
raw_list.status_code
print(raw_list)
list_page = raw_list.content

In [ ]:
#soup
list_bsp = BeautifulSoup(list_page, 'html.parser')
list_bsp

SCRAPING URLS SUBPAGES NORD

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

('https://www.assureurs.pro/entreprise_localisation/toutes-les-regions/nord-pas-de-calais/nord/')

# Find all "a" elements on the page
list_links = driver.find_elements(By.TAG_NAME, 'a')

# Create an empty list to store the href attributes
list_url = []

# Iterate through each anchor element

for link in list_links:
    # Get the href attribute and append it to the list_url
    href = link.get_attribute('href')
    if href == None or link == None :
        continue
    else :
        list_url.append(href)


driver.quit()

# Print or use the list_url as needed
print(list_url)




In [ ]:
list_url_pd = pd.DataFrame({'URL': list_url})
# Save the DataFrame to a CSV file
csv_filename = 'output_urls_AllNord.csv'
list_url_pd.to_csv(csv_filename, index=True)


SCRAPINGS SUBPAGES

In [ ]:
import re

def extract_insurers_lists_as_dataframe(url):
    try:
        #only apply this function to url who contains adresses 
        #https://www.assureurs.pro/entreprise_localisation/toutes-les-regions/nord-pas-de-calais/nord/
        #optionally followed by all of a literal /, optionally followed by a literal home, optionally followed by a literal ? and any number of other characters.
        patternUrl = r"^https:\/\/www\.assureurs\.pro\/entreprise_localisation\/toutes-les-regions\/nord-pas-de-calais\/nord\/([^']+)\/"
        matchUrl = re.search(patternUrl, url)
        if matchUrl:
            page = rq.get(url)
            page.raise_for_status()
            soup = BeautifulSoup(page.content, 'html.parser')

            #extract artcile within targetted url
            
            elements = soup.find_all('article', onclick=True)
            print(elements)
            

            # Extract the URL using regular expression
            if elements:
                for element in elements:
                    onclick_value = element['onclick'] #following BS list created including onclick
                    # Use re.search to find the pattern in the onclick_value
                    pattern = r"window\.location\.href\s*=\s*'([^']+)'" 
                    match = re.search(pattern, onclick_value)
                    
                    
                    if match:
                        url2 = match.group(1) #(1)?
                        print(f'Extracted URL: {url2}')
                    # create a dic with basic info (.text.strip())

                        
                        #info = {
                        #'onclick': url2,
                        #'name': [item.text.strip() if item else "Aucun titre" for item in soup.select('.tile.tile-entreprise a')],
                        #'address': [item.text.strip() if item else "Aucune description" for item in soup.select('.tile.tile-entreprise .tile__body')],
                        #}

                        #'onclick': url2,
                        #'name': soup.select('.tile.tile-entreprise a') if soup.select('.tile.tile-entreprise a') else "Aucun titre",
                        #'adress': soup.select('.tile.tile-entreprise .tile__body') if soup.select('.tile.tile-entreprise .tile__body') else "Aucune adresse"
                    
                    #data.append(info)

                    # Select all elements "div" inside "articles" that match the specified class within information
                        tiles = soup.select('.tile.tile-entreprise')

                        # Create a list to store dictionaries
                        text_div_list = []

                    # Loop through each tile element and extract data
                    for index, tile in enumerate(tiles):
                        onclick_value = url2
                        name_values = [a.text for a in tile.select('a')] or "Aucun titre"
                        address_values = [body.text for body in tile.select('.tile__body')] or "Aucune adresse"

                        # Fill dictionary for the current set of data
                        data_dict = {
                            'url': onclick_value,
                            'name': name_values,
                            'address': address_values
                        }

                        # Append the dictionary to the list
                        text_div_list.append(data_dict)

                        
                        # Display the resulting list of dictionaries
                        for data_dict in text_div_list:
                            print(data_dict)

                    else: #(if no match)
                        print('No URL found in onclick attribute.')
                        continue
                    
                df = pd.DataFrame(text_div_list) 
                return df  
            else:   
                print('Element with onclick attribute not found.')
        else: 
            print('No matched URL')
    except rq.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération de la page : {e}")
        #return pd.DataFrame()

# path CSV file including urls
file_path = 'C:\\Users\\BGE\\ownCloud\\Arnaud\\DEVELOPPEMENT\\ACTIVITE\\Sponso owncloud perso AS\\Scraping\\Assu_Nord_012024\\output_urls_AllNord.csv'

# Read CSV file including urls
urls_df = pd.read_csv(file_path, header=None)

# Apply function on every URL and concatenate résults
all_data_frames = [extract_insurers_lists_as_dataframe(url) for url in urls_df[1]]
print(all_data_frames)

result_df = pd.concat(all_data_frames, ignore_index=False)

# Afficher ou sauvegarder le DataFrame résultant
result_df
# result_df.to_csv('chemin_pour_sauvegarder.csv')  # Décommenter pour sauvegarder dans un fichier CSV


Clean datas and split columns

In [ ]:
#cleaning
insurers_list = result_df.applymap(lambda x: ''.join(x).strip())

#splitting 
# Create 'insurers_list_addresses' DataFrame
insurers_list_addresses = pd.DataFrame()
insurers_list_addresses[['street', 'zipcode and city']] = insurers_list['address'].str.split('\n', expand=True)

insurers_list_cities= pd.DataFrame()
insurers_list_cities[['zipcode','city']] = insurers_list_addresses['zipcode and city'].str.split(' ', expand=True)
insurers_list_cities

#concatening
insurers= pd.DataFrame()
insurers = [insurers_list, insurers_list_addresses, insurers_list_cities]
insurers_list_vf = pd.concat(insurers, axis=1)


In [ ]:
# Set the index with the number of the line as a unique identifier
insurers_list_vf.index = range(1, len(insurers_list_vf) + 1)
insurers_list_vf

In [ ]:
insurers_list_vf = insurers_list_vf[:][['url', 'name', 'street', 'zipcode', 'city']]
insurers_list_vf = insurers_list_vf.rename_axis('ID')
insurers_list_vf.sort_values('city')

In [ ]:
#erase "None" address 
import numpy as np
insurers_list_vf['street'].replace('Aucune adresse', np.nan)
insurers_list_vf.dropna()

In [ ]:
insurers_list_vf.groupby('city').size()

In [ ]:
result_df.applymap(lambda x: str(x).strip())
csv_filename = 'outputs_lists_insurers_AllNord.csv'
insurers_list_vf.to_csv(csv_filename)


SELENIUM / WEBDRIVER - send form response

In [ ]:
#webdriverwait and Explicit COnditions 

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:


# Set up the WebDriver (adjust the path based on your browser)
from selenium.webdriver.chrome.service import Service
service = Service(executable_path='C:\\Users\\BGE\\chromedriver_win32\\chromedriver.exe')

#instance webdriver

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

#requests looping list of pages including forms
urls_forms = list(insurers_list_vf['url'])
for url_forms in urls_forms:
    driver.get(url_forms)

    # Use WebDriverWait to wait for the form elements to be present 
    try:
        element = WebDriverWait(driver, 10).until(
            # box welcoming text to express the need
            EC.presence_of_element_located((By.NAME, 'input_9')),
            #firstname
            EC.presence_of_element_located((By.NAME, 'input_11')),
            #name
            EC.presence_of_element_located((By.NAME, 'input_10')),
            #address
            EC.presence_of_element_located((By.NAME, 'input_2.1')),
            #zipcode
            EC.presence_of_element_located((By.NAME, 'input_2.5')),
            #city
            EC.presence_of_element_located((By.NAME, 'input_2.3')),
            #email
            EC.presence_of_element_located((By.NAME, 'input_3')),
            #phone
            EC.presence_of_element_located((By.NAME, 'input_4'))
        )


        # Fill the form (replace with the actual form elements and data)
        driver.find_element(By.NAME, 'input_9').send_keys('Your data')
        driver.find_element(By.NAME, 'input_11').send_keys('Your data')
        driver.find_element(By.NAME, 'input_10').send_keys('Your data')
        driver.find_element(By.NAME, 'input_2.1').send_keys('Your data')
        driver.find_element(By.NAME, 'input_2.5').send_keys('Your data')
        driver.find_element(By.NAME, 'input_2.3').send_keys('Your data')
        driver.find_element(By.NAME, 'input_3').send_keys('Your data')
        driver.find_element(By.NAME, 'input_4').send_keys('Your data')

    except Exception as e:
        print(f"An error occurred: {e}")
        continue  # Move to the next URL if an error occurs

    # Fill the form (replace with the actual form elements and data)
    driver.find_element(By.NAME, 'input_9').send_keys('Your data')
    driver.find_element(By.NAME, 'input_11').send_keys('Your data')
    driver.find_element(By.NAME, 'input_10').send_keys('Your data')
    driver.find_element(By.NAME, 'input_2.1').send_keys('Your data')
    driver.find_element(By.NAME, 'input_2.5').send_keys('Your data')
    driver.find_element(By.NAME, 'input_2.3').send_keys('Your data')
    driver.find_element(By.NAME, 'input_3').send_keys('Your data')
    driver.find_element(By.NAME, 'input_4').send_keys('Your data')

    # Submit the form (replace with the actual submit action)
    #driver.find_element(By.ID, 'gform_submit_button_1').click()